<a href="https://colab.research.google.com/github/fernandolievano/ProyectoFinal_PP1/blob/main/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from urllib.request import urlopen
from IPython.display import HTML

RAW_URL = "https://raw.githubusercontent.com/fernandolievano/ProyectoFinal_PP1/main/assets/doc.html"
with urlopen(RAW_URL) as f:
    html = f.read().decode("utf-8", errors="replace")
HTML(html)

# Preparación del entorno y dataset

In [18]:
%pip install pandas numpy plotly

In [19]:
# importación de librerías a utilizar
import pandas as pd
import plotly.express as px

In [20]:
# importación de dataset
dataset_url = "https://raw.githubusercontent.com/fernandolievano/ProyectoFinal_PP1/main/data/stack-overflow-developer-survey-2025.csv.gz"
df = pd.read_csv(dataset_url, compression='gzip')  # especifico que estoy usando un archivo .gz

print("✅ DataSet cargado con éxto")
print(f"ℹ️ Forma del DataSet: {df.shape}")
print("ℹ️ Información del DataSet")
df.info()

✅ DataSet cargado con éxto
ℹ️ Forma del DataSet: (49123, 170)
ℹ️ Información del DataSet
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49123 entries, 0 to 49122
Columns: 170 entries, ResponseId to JobSat
dtypes: float64(50), int64(1), object(119)
memory usage: 63.7+ MB


### Utils

In [21]:
# Funciones Reutilizables

def contar_valores_explodidos(dataframe, columna, separador):
    """
    Cuenta las incidencias de valores en una columna que contiene strings
    separados por un delimitador.
    :param dataframe:
    :param columna:
    :param separador:
    :return:
        Una Serie de pandas
    """
    if columna not in dataframe.columns:
        raise ValueError(f"Columna {columna} no encontrada")

    return (dataframe[columna]
            .str.split(separador)
            .explode()  # cada elemento de la lista se vuelve una fila
            .value_counts())


# Análisis Exploratorio de Datos
Llevaremos a cabo un análisis exploratorio de datos utilizando visualizaciones interactivas y estadísticas descriptivas, con el objetivo de comprender mejor los resultados de la encuesta hecha por Stack Overflow en 2025. Mediante este proceso podremos responder preguntas sobre lenguajes de programación más usados, tecnologías en tendencias, roles comunes y niveles salariales.

## Verificación de nulos

In [22]:
pd.set_option('display.max_rows', None)  # Activo display.max_rows para poder ver todos mis campos nulos

tiene_nulos = df.isnull().sum() > 0  # Máscara para mostrar solo resultados con valores nulos
nulos = df.isnull().sum()[tiene_nulos].sort_values(ascending=False)
nulos_cantidad = len(nulos)
nulos_porcentaje = (nulos / df.shape[0]) * 100

print(f"❗ Encontramos {nulos_cantidad} campos con datos nulos")
print("ℹ️ Porcentaje de valores nulos encontrados por campo:")

print(nulos_porcentaje.sort_values(ascending=False).to_frame(name="Porcentaje"))

pd.reset_option('display.max_rows', None)  # Desactivo de nuevo display.max_rows

❗ Encontramos 167 campos con datos nulos
ℹ️ Porcentaje de valores nulos encontrados por campo:
                                          Porcentaje
AIAgentObsWrite                            99.462574
SOTagsWant Entry                           99.124646
SOTagsHaveEntry                            99.069682
AIModelsWantEntry                          99.035075
AIAgentOrchWrite                           99.028968
JobSatPoints_15_TEXT                       98.648291
AIAgentKnowWrite                           98.442685
AIModelsHaveEntry                          98.422328
SO_Actions_15_TEXT                         98.324614
AIAgentExtWrite                            98.253364
CommPlatformWantEntr                       97.591759
CommPlatformHaveEntr                       96.997333
DatabaseWantEntry                          96.889441
OfficeStackWantEntry                       96.708263
TechOppose_15_TEXT                         96.651263
TechEndorse_13_TEXT                        95.912302
DevE

### Altos nulos
En este gráfico se muestran las variables con mayor porcentaje de valores faltantes.
Dado que superan ampliamente el 80% de nulos, pueden considerarse prescindibles para el análisis, ya que su nivel de completitud es insuficiente para obtener insights confiables.

In [23]:
sobre_80_porciento = nulos_porcentaje > 80  # Máscara obtener resultados mayores a 80%
nulos_altos = nulos_porcentaje[sobre_80_porciento].sort_values(ascending=False)

# Gráfico
fig_altos_nulos = px.bar(
    nulos_altos,
    color='value',
    orientation='h',
    title='⚠️ Variables con más del 80% de valores nulos',
    labels={'value': 'Porcentaje de Nulos', 'index': 'Variable'},
    color_continuous_scale='Reds'
)
fig_altos_nulos.update_layout(
    height=40 * len(nulos_altos),
    font=dict(
        size=14,
        color='#fff',
    ),
    title=dict(
        xanchor='center',
        x=0.5
    ),
    paper_bgcolor='black',
    plot_bgcolor='black',
)
fig_altos_nulos.show()

### Nulos en variables clave
Se muestran variables fundamentales del análisis (como país, educación y años de experiencia) que presentan un bajo porcentaje de valores faltantes, lo que permite tratarlos sin afectar la calidad de los resultados.

In [24]:
variables_clave = ['Country', 'Employment', 'EdLevel', 'WorkExp', 'YearsCode', 'ConvertedCompYearly']
nulos_claves = nulos[variables_clave].sort_values(ascending=False)

# Gráfico
fig_nulos_clave = px.bar(
    nulos_claves,
    title='🔑 Cantidad de nulos en variables clave',
    labels={'value': 'Cantidad de Nulos', 'index': 'Variable'},
    color='value',
    color_continuous_scale='sunsetdark'
)
fig_nulos_clave.update_layout(
    font=dict(
        size=14,
        color='#fff',
    ),
    title=dict(
        xanchor='center',
        x=0.5
    ),
    paper_bgcolor='black',
    plot_bgcolor='black',
)
fig_nulos_clave.show()

 # Análisis de hipótesis
 En esta sección se abordan las preguntas planteadas al inicio del proyecto, utilizando gráficos interactivos y resúmenes numéricos para explorarlas en detalle.

## 1. ¿Qué lenguajes de programación están en auge?

In [25]:
# Usamos la función "contar_valores_explodidos" para dividir los valores
# de la columna 'LanguageHaveWorkedWith' (separados por ';') y obtener
# un conteo de los lenguajes más utilizados.
lenguajes_usados_exploded = contar_valores_explodidos(dataframe=df, columna='LanguageHaveWorkedWith', separador=';')
lenguajes_usados_exploded = lenguajes_usados_exploded.sort_values(ascending=True)

fig_lenguajes_usados = px.bar(
    lenguajes_usados_exploded,
    title='👨‍💻 Lenguajes de programación más utilizados en 2025',
    color='value',
    color_continuous_scale='pinkyl',
    orientation='h',
    labels={'value': 'Uso', 'LanguageHaveWorkedWith': 'Lenguaje de Programación'},
)
fig_lenguajes_usados.update_layout(
    font=dict(
        size=14,
        color='#fff',
    ),
    paper_bgcolor='black',
    plot_bgcolor='black',
    height=36 * len(lenguajes_usados_exploded),
    title=dict(
        xanchor='center',
        x=0.5
    ),
)

fig_lenguajes_usados.show()

In [26]:
# Contamos cuántos desarrolladores indicaron que desean trabajar con cada lenguaje.
lenguajes_deseados = contar_valores_explodidos(
    dataframe=df,
    columna='LanguageWantToWorkWith',
    separador=';'
)

# Calculamos el número total de encuestados (denominador común para normalizar).
n_encuestados = len(df)

# Convertimos los conteos a porcentajes sobre el total de encuestados.
lenguajes_deseados_pct = (lenguajes_deseados / n_encuestados) * 100
lenguajes_usados_pct = (lenguajes_usados_exploded / n_encuestados) * 100

# Calculamos el "auge" como la diferencia entre deseo y uso (% de encuestados).
#  > 0  → más deseado que usado (auge)
#  = 0  → estable
#  < 0  → más usado que deseado (retroceso)
lenguajes_auge = lenguajes_deseados_pct - lenguajes_usados_pct
lenguajes_auge = lenguajes_auge[lenguajes_auge > 0]
lenguajes_auge = lenguajes_auge.sort_values(ascending=True)


# Graficamos
fig_lenguajes_auge = px.bar(
    lenguajes_auge,
    title='📈 Lenguajes de Programación en Auge en 2025',
    color='value',
    color_continuous_scale='tempo',
    orientation='h',
    labels={'index': 'Lenguajes de Programación', 'value': '% Auge'}
)
fig_lenguajes_auge.update_layout(
    font=dict(
        size=14,
        color='#fff',
    ),
    paper_bgcolor='black',
    plot_bgcolor='black',
    height=36 * len(lenguajes_auge),
    title=dict(
        xanchor='center',
        x=0.5
    ),
)


## 2. ¿Qué tecnologías muestran menor interés futuro?

In [27]:
# Calculamos la diferencia entre el porcentaje de deseo y el porcentaje de uso.
# Un valor negativo indica que la tecnología es utilizada por más desarrolladores
# de los que manifiestan querer usarla en el futuro → "más usada que deseada".
lenguajes_retroceso = lenguajes_deseados_pct - lenguajes_usados_pct
lenguajes_retroceso = lenguajes_retroceso[lenguajes_retroceso < 0].sort_values(ascending=False)

# Graficamos estas tecnologías ordenadas, para visualizar aquellas que muestran
# menor interés futuro en comparación con su nivel de uso actual.
fig_lenguajes_retroceso = px.bar(
    lenguajes_retroceso,
    title="📉 Tecnologías que pierden interés en 2025",
    color='value',
    color_continuous_scale='pinkyl_r',
    orientation='h',
    labels={'index': 'Lenguajes de Programación', 'value': '% Retroceso'}
)
fig_lenguajes_retroceso.update_layout(
    font=dict(size=14, color="#fff"),
    paper_bgcolor="black",
    plot_bgcolor="black",
    height=36 * len(lenguajes_retroceso),
    title=dict(xanchor="center", x=0.5),
)

fig_lenguajes_retroceso.show()

> ⚠️ Un valor negativo significa que un lenguaje es más usado que deseado, lo cual no siempre significa que un lenguaje esté "muriendo", sino que en muchos casos refleja tecnologías muy **consolidadas**.